In [24]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Flatten
from keras.layers import Dense, Embedding, LSTM, Dropout, Activation
from keras import backend as K
from deepexplain.tensorflow import DeepExplain
import pandas as pd
from keras.models import load_model
from keras.models import model_from_json

In [2]:
LSTM_MODEL_JSON = '../saved_model/model_lstm.json'
LSTM_MODEL_WEIGHTS = '../saved_model/model_lstm.h5'
HISTORY_FILE = '../saved_model/history_lstm.json'

In [3]:
def save_lstm_model(model):
    # load json and create model
    model_json = model.to_json()
    with open(LSTM_MODEL_JSON, 'w') as jsonfile:
        jsonfile.write(model_json)
    # serialize weights to HDF5
    model.save_weights(LSTM_MODEL_WEIGHTS)

def load_lstm_model(model):
    # load weights into new model
    loaded_model = model_from_json(LSTM_MODEL_JSON)
    loaded_model.load_weights(LSTM_MODEL_WEIGHTS)
    # evaluate loaded model on test data
    return loaded_model

In [4]:
df = pd.read_csv('../data/final_data_less.csv')

In [5]:
t=Tokenizer()
t.fit_on_texts(df['clean_sentiment'])

In [6]:
vocab_size = len(t.word_index)+1
encoded_docs = t.texts_to_sequences(df['clean_sentiment'])
print(vocab_size)

7051


In [12]:
labels = pd.get_dummies(df['sentiment'].values)
print(labels.shape)

(1000, 2)


In [13]:
max_length = max([len(x) for x in encoded_docs])
print(max_length)

580


In [14]:
# padding
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [15]:
split_fraction = 0.8
split_idx = int(len(padded_docs)*split_fraction)
print(split_idx)

X_train, X_test = padded_docs[:split_idx], padded_docs[split_idx:]
y_train, y_test = labels[:split_idx], labels[split_idx:]

print("Training Shape: ", X_train.shape, "== Train Lables: ", y_train.shape)
print("Test Shape: ", X_test.shape, "== Test Lables: ", y_test.shape)

800
Training Shape:  (800, 580) == Train Lables:  (800, 2)
Test Shape:  (200, 580) == Test Lables:  (200, 2)


In [18]:
model = Sequential()
model.add(Embedding(vocab_size,128,input_length=max_length))
model.add(Flatten());
model.add(Dense(100, activation='relu')); # input_shape=(max_words,)
model.add(Dropout(0.5));
model.add(Dense(2, activation='softmax'));
# model.add(Dense(4, activation='linear'));
model.add(Activation('softmax'));
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']);
print(model.summary());

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 580, 128)          902528    
_________________________________________________________________
flatten_3 (Flatten)          (None, 74240)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               7424100   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 202       
Total params: 8,326,830
Trainable params: 8,326,830
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(X_test, y_test),
          verbose=1,
          shuffle=True);

# predict on test data
y_pred = model.predict(np.array(X_test));
y_test = np.array(y_test);

Train on 800 samples, validate on 200 samples
Epoch 1/5
800/800 [==============================] - 4s 5ms/step - loss: 0.0136 - acc: 0.9988 - val_loss: 0.5297 - val_acc: 0.7300
Epoch 2/5
800/800 [==============================] - 4s 5ms/step - loss: 0.0058 - acc: 0.9988 - val_loss: 0.5343 - val_acc: 0.7400
Epoch 3/5
800/800 [==============================] - 4s 5ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.5421 - val_acc: 0.7550
Epoch 4/5
800/800 [==============================] - 4s 5ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.5423 - val_acc: 0.7350
Epoch 5/5
800/800 [==============================] - 4s 5ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.5481 - val_acc: 0.7300


In [20]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 74.50%


In [21]:
save_lstm_model(model)

In [22]:
current_session = K.get_session()

In [ ]:
with DeepExplain(session=current_session) as de:  # <-- init DeepExplain context
    model = Sequential()
    model.add(Embedding(vocab_size,128,input_length=max_length))
    model.add(Flatten());
    model.add(Dense(100, activation='relu')); # input_shape=(max_words,)
    model.add(Dropout(0.5));
    model.add(Dense(2, activation='softmax'));
    # model.add(Dense(4, activation='linear'));
    model.add(Activation('softmax'));
    model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']);
    print(model.summary());
    model.fit(X_train, y_train,
          batch_size=10,
          epochs=5,
          validation_data=(X_test, y_test),
          verbose=1,
          shuffle=True);

    # predict on test data
    y_pred = model.predict(np.array(X_test));
    y_test = np.array(y_test);
    
    # Evaluate the embedding tensor on the model input (in other words, perform the lookup)
    embedding_tensor = model.layers[0].output
    input_tensor = model.inputs[0]
    embedding_out = current_session.run(embedding_tensor, {input_tensor: X_test});

    xs = X_test;
    ys = y_test;
    # Run DeepExplain with the embedding as input
    attributions = de.explain('elrp', model.layers[-2].output * ys, model.layers[1].input, embedding_out);
    print("attributions shape --- {}".format(attributions.shape));

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 580, 128)          902528    
_________________________________________________________________
flatten_6 (Flatten)          (None, 74240)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               7424100   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 202       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 8,326,830
Trainable params: 8,326,830
Non-trainable params: 0
_________________________________________________________________
